In [0]:
import random

def generate_complex_query():
    base_query = """
SELECT
    NULL AS EmployeeID,
    NULL AS Name,
    d.Name AS Department,
    CONCAT('Department Total: ', CAST(ds.TotalDeptSalary AS VARCHAR)) AS SalaryStatus,
    ds.NumEmployees AS ProjectsCompleted,
    NULL AS HireYear,
    NULL AS HireMonth,
    NULL AS LastProjectYear,
"""

    # Generate 599 columns, each randomly a literal or a CASE with subquery
    cols = []
    for i in range(1, 600):
        rnd = random.random()
        if rnd < 0.05:
            col = (
                f"CASE WHEN ds.TotalDeptSalary > 100000 AND ds.TotalDeptSalary < 200000 AND NVL(a.emp_class, 'N') NOT IN ('A', 'B') -- story 1234\n"
                f"THEN NVL((SELECT COUNT(*) FROM Employees WHERE DepartmentID = ds.DepartmentID),0)\n"
                f"ELSE 0 END as col{i}"
            )
        elif rnd >= 0.05 and rnd < 0.1:
            col = (
                f"CASE WHEN ds.TotalDeptSalary > 100000 AND ds.TotalDeptSalary < 200000 AND NVL(a.emp_class, 'N') NOT IN ('A', 'B') -- story 1234\n"
                f"THEN NVL((SELECT COUNT(*) FROM Employees WHERE DepartmentID = ds.DepartmentID),2)\n"
                f"ELSE 0 END as col{i}"
            )
        elif rnd >= 0.1 and rnd < 0.2:
            col = (
                f"CASE WHEN ds.TotalDeptSalary > 100000 AND ds.TotalDeptSalary < 200000 AND NVL(a.emp_class, 'N') NOT IN ('A', 'B') -- story 1234\n"
                f"THEN NVL((SELECT COUNT(*) FROM Employees WHERE DepartmentID = ds.DepartmentID),'x')\n"
                f"ELSE 0 END as col{i}"
            )
        else:
            col = f"'col{i}' as col{i}"
        cols.append(col)
    cols_str = ",\n".join(cols)

    # Assemble the final query
    final_query = f"""{base_query}{cols_str}
FROM
    DeptStats ds
    JOIN Departments d ON ds.DepartmentID = d.DepartmentID;
"""
    return final_query

query_string = generate_complex_query()
print(query_string)


In [0]:
result = dbutils.notebook.run(
    "../parser_driver", 
    0, 
    {
        "sql": query_string, 
        "endpoint":"databricks-claude-sonnet-4",
        # "endpoint": "",
        "parse_log_table": "users.paul_signorelli.sql_parsing_log",
        "column_chunk_size": "50",
        "total_stages": "3"
    }
)